In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score

In [12]:
# === 1. Wczytaj dane ===
df = pd.read_csv('final_dataset.csv', index_col='Date', parse_dates=True)

In [13]:
# === 2. Wybierz cechy i etykietę ===
features = ['Otwarcie', 'Najwyzszy', 'Najnizszy', 'Wolumen', '1 Mo', '2 Mo', '3 Mo', '4 Mo', '6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr', '7 Yr', '10 Yr', '20 Yr', '30 Yr', 'Daily index price Change (%)', 'STS', 'MACD', 'MACD_Signal', 'RSI', 'EMA_14', 'EMA_50', 'EMA_200', 'SMA_14', 'SMA_50', 'SMA_200']  # <-- Dostosuj do swoich danych
target = 'Label'  # <-- Dostosuj nazwę targetu (zmiana na 'Label' z Twojego przykładu)

X = df[features]
y = df[target]

In [14]:
# === 3. Skalowanie danych ===
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [15]:
# === 4. Podział na zbiory treningowy/testowy ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, shuffle=False
)

In [16]:
# === 5. LSTM Model ===
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Reshape for LSTM
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))  # Reshape for LSTM

# LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))  # Binary classification

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)


Epoch 1/10


C:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4701 - loss: nan
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4607 - loss: nan
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4635 - loss: nan
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4639 - loss: nan
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4695 - loss: nan
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4652 - loss: nan
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4666 - loss: nan
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4677 - loss: nan
Epoch 9/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4708 - loss: nan
Epoch 10/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4683 - loss: nan


In [17]:
# === 6. Predykcje ===
y_pred_lstm_scaled = lstm_model.predict(X_test_lstm)
y_pred_lstm = (y_pred_lstm_scaled > 0.5).astype(int)  # Threshold dla klasyfikacji binarnej


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [18]:
# === 7. Przywrócenie oryginalnej skali ===
y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_lstm_inv = scaler_y.inverse_transform(y_pred_lstm)


In [19]:
# === 8. Analiza wyników ===
print("LSTM Classification Report:")
print(classification_report(y_test_inv, y_pred_lstm_inv))
accuracy_lstm = accuracy_score(y_test_inv, y_pred_lstm_inv)
print(f"LSTM Accuracy: {accuracy_lstm:.2f}")

LSTM Classification Report:
              precision    recall  f1-score   support

         0.0       0.46      1.00      0.63       796
         1.0       0.00      0.00      0.00       953

    accuracy                           0.46      1749
   macro avg       0.23      0.50      0.31      1749
weighted avg       0.21      0.46      0.28      1749

LSTM Accuracy: 0.46


C:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo